In [5]:
import pandas as pd
import os
from tqdm import tqdm

input_dir = 'D:/Data_Analyst/Python_Advanced_Data/Final Capstone Project/Datasets'
files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.xlsx', '.xls')) and not f.startswith('~$')]

def extract_transform(file_name):
    full_path = os.path.join(input_dir, file_name)
    df_raw = pd.read_excel(full_path, header=None)
    
    header_idx = 0
    for i, row in df_raw.iterrows():
        row_str = row.astype(str).values
        if any('jan' in s.lower() for s in row_str) or any('no.' in s.lower() for s in row_str):
            header_idx = i
            break
            
    df = pd.read_excel(full_path, skiprows=header_idx)
    df.rename(columns={df.columns[0]: 'No.', df.columns[1]: 'Label'}, inplace=True)
    
    # Extract clean file name
    df['File Name'] = file_name.replace('.xlsx', '').replace('.xls', '')
    
    df1 = pd.melt(df, id_vars=["No.", "Label", "File Name"], var_name="Month_Year", value_name="Amount")
    df1 = df1[df1['Month_Year'].astype(str).str.contains(' ', na=False)]
    
    def split_month_year(val):
        val_str = str(val).strip()
        parts = val_str.split(' ')
        if len(parts) >= 2:
            return parts[0], parts[1]
        return None, None

    df1[['Month', 'Year']] = df1['Month_Year'].apply(lambda x: pd.Series(split_month_year(x)))
    
    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    
    df1['Month'] = df1['Month'].str.capitalize().map(month_map)
    df1 = df1.dropna(subset=['Month', 'Year', 'Amount'])

    # Convert to numeric types
    df1['Month'] = df1['Month'].astype(int)
    df1['Year'] = df1['Year'].astype(int)
    df1['Amount'] = pd.to_numeric(df1['Amount'], errors='coerce')
    df1 = df1.dropna(subset=['Amount'])
    
    df1['Label'] = df1['Label'].astype(str).str.strip()
    df1 = df1.drop(columns=['Month_Year'])
    
    return df1

# --- MAIN LOOP ---
all_dfs = []
for file in tqdm(files, desc="Processing"):
    try:
        processed_data = extract_transform(file)
        if not processed_data.empty:
            all_dfs.append(processed_data)
    except Exception as e:
        print(f"Error on {file}: {e}")

if all_dfs:
    # 1. Combine all dataframes
    master_df = pd.concat(all_dfs, ignore_index=True)
    
    # 2. Sort by File Name first, then Year/Month to keep data organized
    master_df = master_df.sort_values(by=['File Name', 'Year', 'Month', 'Label']).reset_index(drop=True)
    
    # 3. Create the Serial Number sequence (1, 2, 3...)
    master_df['No.'] = range(1, len(master_df) + 1)
    
    # 4. REORDER COLUMNS: Shift 'File Name' to the LAST position
    # We list all columns except 'File Name', then add 'File Name' at the end
    cols_at_end = ['File Name']
    other_cols = [c for c in master_df.columns if c not in cols_at_end]
    master_df = master_df[other_cols + cols_at_end]
    
    # 5. Save to Excel
    master_df.to_excel("Master_Combined_Dataset.xlsx", index=False)
    
    print(f"\nSuccess! Total Rows: {len(master_df)}")
    print("Column Order: [No.] -> [Label] -> [Amount] -> [Month] -> [Year] -> [File Name]")
else:
    print("\nNo data found.")

Processing: 100%|████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.17s/it]



Success! Total Rows: 25913
Column Order: [No.] -> [Label] -> [Amount] -> [Month] -> [Year] -> [File Name]
